In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import glob, pylab, pandas as pd
import pydicom, numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pylab as plt
import os
import seaborn as sns
import warnings
warnings.filterwarnings(action='once')
import pickle
from tqdm import tqdm, tqdm_notebook
%load_ext autoreload
%autoreload 2
%matplotlib notebook
from helper import *
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
data_dir = '/media/nvme/data/RSNA/'
train_images_dir=data_dir+'stage_1_train_images/'
test_images_dir=data_dir+'stage_1_test_images/'

In [65]:
train_base_df = pd.read_csv(data_dir+'stage_1_train.csv')

In [67]:
train_base_df.shape[0]

4045572

In [4]:
train_base_df['Sub_type'] = train_base_df['ID'].str.split("_", n = 3, expand = True)[2]
train_base_df['PatientID'] = train_base_df['ID'].str.split("_", n = 3, expand = True)[1]
train_base_df.head()

,ID,Label,Sub_type,PatientID
0,ID_28fbab7eb_epidural,0.5,epidural,28fbab7eb
1,ID_28fbab7eb_intraparenchymal,0.5,intraparenchymal,28fbab7eb
2,ID_28fbab7eb_intraventricular,0.5,intraventricular,28fbab7eb
3,ID_28fbab7eb_subarachnoid,0.5,subarachnoid,28fbab7eb
4,ID_28fbab7eb_subdural,0.5,subdural,28fbab7eb


In [26]:
sub_types=train_base_df.Sub_type.unique()
sub_types

array(['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid',
       'subdural', 'any'], dtype=object)

In [27]:
dfs =[]
for sub_type in tqdm_notebook(sub_types):
    df = train_base_df[train_base_df.Sub_type==sub_type][['PatientID','Label']].copy()
    df=df.rename(columns={"Label": sub_type}).reset_index(drop=True)
    dfs.append(df)
train_df=dfs[0]
for df in tqdm_notebook(dfs[1:]):
    train_df=test_df.merge(df,on='PatientID')

In [11]:
PID = np.zeros(train_df.shape[0],dtype=object)
StudyI = np.zeros(train_df.shape[0],dtype=object)
SeriesI = np.zeros(train_df.shape[0],dtype=object)
WindowCenter = np.zeros(train_df.shape[0],dtype=object)
WindowWidth = np.zeros(train_df.shape[0],dtype=object)
ImagePositionX = np.zeros(train_df.shape[0],dtype=np.float)
ImagePositionY = np.zeros(train_df.shape[0],dtype=np.float)
ImagePositionZ = np.zeros(train_df.shape[0],dtype=np.float)

for i,row in tqdm_notebook(train_df.iterrows(),total=train_df.shape[0]):
    ds = pydicom.dcmread(train_images_dir + 'ID_{}.dcm'.format(row['PatientID']))
    SeriesI[i]=ds.SeriesInstanceUID
    PID[i]=ds.PatientID
    StudyI[i]=ds.StudyInstanceUID 
    WindowCenter[i]=ds.WindowCenter
    WindowWidth[i]=ds.WindowWidth
    ImagePositionX[i]=float(ds.ImagePositionPatient[0])
    ImagePositionY[i]=float(ds.ImagePositionPatient[1])
    ImagePositionZ[i]=float(ds.ImagePositionPatient[2])
train_df['SeriesI']=SeriesI
train_df['PID']=PID
train_df['StudyI']=StudyI
train_df['WindowCenter']=WindowCenter
train_df['WindowWidth']=WindowWidth
train_df['ImagePositionZ']=ImagePositionZ
train_df['ImagePositionX']=ImagePositionX
train_df['ImagePositionY']=ImagePositionY


In [ ]:
train_df.to_csv(data_dir+'train.csv',index=False)

In [3]:
sample_submission=pd.read_csv(data_dir+'stage_1_sample_submission.csv')
sample_submission.head()

,ID,Label
0,ID_28fbab7eb_epidural,0.5
1,ID_28fbab7eb_intraparenchymal,0.5
2,ID_28fbab7eb_intraventricular,0.5
3,ID_28fbab7eb_subarachnoid,0.5
4,ID_28fbab7eb_subdural,0.5


In [4]:
test_base_df=sample_submission.copy()
test_base_df['Sub_type'] = test_base_df['ID'].str.split("_", n = 3, expand = True)[2]
test_base_df['PatientID'] = test_base_df['ID'].str.split("_", n = 3, expand = True)[1]
test_base_df.head()

,ID,Label,Sub_type,PatientID
0,ID_28fbab7eb_epidural,0.5,epidural,28fbab7eb
1,ID_28fbab7eb_intraparenchymal,0.5,intraparenchymal,28fbab7eb
2,ID_28fbab7eb_intraventricular,0.5,intraventricular,28fbab7eb
3,ID_28fbab7eb_subarachnoid,0.5,subarachnoid,28fbab7eb
4,ID_28fbab7eb_subdural,0.5,subdural,28fbab7eb


In [234]:
test_ids=test_df.PatientID.unique()

In [26]:
sub_types=test_base_df.Sub_type.unique()
sub_types

array(['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid',
       'subdural', 'any'], dtype=object)

In [7]:
dfs =[]
for sub_type in tqdm_notebook(sub_types):
    df = test_base_df[test_base_df.Sub_type==sub_type][['PatientID','Label']].copy()
    df=df.rename(columns={"Label": sub_type}).reset_index(drop=True)
    dfs.append(df)
test_df=dfs[0]
for df in tqdm_notebook(dfs[1:]):
    test_df=test_df.merge(df,on='PatientID')

In [8]:
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5


In [11]:
PID = np.zeros(test_df.shape[0],dtype=object)
StudyI = np.zeros(test_df.shape[0],dtype=object)
SeriesI = np.zeros(test_df.shape[0],dtype=object)
WindowCenter = np.zeros(test_df.shape[0],dtype=object)
WindowWidth = np.zeros(test_df.shape[0],dtype=object)
ImagePositionX = np.zeros(test_df.shape[0],dtype=np.float)
ImagePositionY = np.zeros(test_df.shape[0],dtype=np.float)
ImagePositionZ = np.zeros(test_df.shape[0],dtype=np.float)

for i,row in tqdm_notebook(test_df.iterrows(),total=test_df.shape[0]):
    ds = pydicom.dcmread(test_images_dir + 'ID_{}.dcm'.format(row['PatientID']))
    SeriesI[i]=ds.SeriesInstanceUID
    PID[i]=ds.PatientID
    StudyI[i]=ds.StudyInstanceUID 
    WindowCenter[i]=ds.WindowCenter
    WindowWidth[i]=ds.WindowWidth
    ImagePositionX[i]=float(ds.ImagePositionPatient[0])
    ImagePositionY[i]=float(ds.ImagePositionPatient[1])
    ImagePositionZ[i]=float(ds.ImagePositionPatient[2])
test_df['SeriesI']=SeriesI
test_df['PID']=PID
test_df['StudyI']=StudyI
test_df['WindowCenter']=WindowCenter
test_df['WindowWidth']=WindowWidth
test_df['ImagePositionZ']=ImagePositionZ
test_df['ImagePositionX']=ImagePositionX
test_df['ImagePositionY']=ImagePositionY


In [23]:
test_df.to_csv(data_dir+'test.csv',index=False)